# A simple Wallet Portfolio Viewer

In [54]:
import os
import requests
import pandas as  pd
import numpy as np 
from dotenv import load_dotenv

In [ ]:
# loading the environment variables from the .env file
load_dotenv ()
DUNE_API_KEY = os.getenv('DUNE_API_KEY')

In [68]:
# Function to get wallet balances from  dune analytics using Dune Sim API

WALLET_ADDRESS = '0xd8da6bf26964af9d7eed9e03e53415d37aa96045' # sample wallet address

def fetch_sim_balances(address):
    url = f"https://api.sim.dune.com/v1/evm/balances/{address}"
    headers = {'X-Sim-Api-Key': DUNE_API_KEY}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise error for bad status
        data = response.json()
        return data.get('balances', [])  # List of balance dicts
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Sim balances: {e}")
        return []


In [69]:
# Function to fetch prices from CoinGecko for tokens on ethereum
def fetch_coingecko_prices(chain, contract_addresses):
    if not contract_addresses:
        return {}
    # CoinGecko platform ID (map chain to platform; simplify to 'ethereum' for now)
    platform = 'ethereum' if chain.lower() == 'ethereum' else None
    if not platform:
        return {}  
    url = f"https://api.coingecko.com/api/v3/simple/token_price/{platform}"
    params = {
        'contract_addresses': ','.join(contract_addresses),
        'vs_currencies': 'usd'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()  # Dict like {'0x': {'usd': price}}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching CoinGecko prices: {e}")
        return {}

In [ ]:
# Fetching balances from Sim

balances = fetch_sim_balances(WALLET_ADDRESS)
if not balances:
    print("No balances found or error occurred.")
else:
    # Creating DataFrame
    df = pd.DataFrame(balances)

    # To handle missing prices: Collect contracts per chain where price_usd is missing or None
    missing_prices = {}
    for _, row in df.iterrows():
        if 'price_usd' not in row or row['price_usd'] is None:
            chain = row['chain']
            addr = row['address'] if row['address'] != 'native' else None
            if addr:
                if chain not in missing_prices:
                    missing_prices[chain] = []
                missing_prices[chain].append(addr.lower())

In [ ]:
# Fetching missing prices from CoinGecko

for chain, address in missing_prices.items():
    prices = fetch_coingecko_prices(chain, address)
    for addr, price_data in prices.items():
        mask = (df['address'].str.lower() == address) & (df['chain'] == chain)
        df.loc[mask, 'price_usd'] = price_data.get('usd')

In [ ]:
# Calculating the value_usd if missing (amount is string, converted to float)

df['amount'] = df['amount'].astype(float)
df['value_usd'] = np.where(
    df['value_usd'].isnull(),
    df['amount'] / (10 ** df['decimals']) * df['price_usd'],
    df['value_usd']
    )

In [ ]:
# Filtering out the zero values
df = df[df['value_usd'] > 0]

# Total portfolio value
total_value = np.sum(df['value_usd'])
print(f"Total Portfolio Value: ${total_value:.2f}")
print(df[['symbol', 'chain', 'value_usd']])

# Saving to JSON
df.to_json('portfolio.json', orient='records', indent=4)
print("Saved to portfolio.json")

Total Portfolio Value: $937634737079254.88
        symbol     chain     value_usd
0       AiMeme  ethereum  8.669824e+14
1        ERBAI  ethereum  2.029522e+13
2        MONEY  ethereum  1.043810e+13
3         USSR  ethereum  8.396683e+12
4       ADCBOT  ethereum  6.780834e+12
..         ...       ...           ...
683        SDR  ethereum  2.127179e-14
684  MAGAnolia  ethereum  1.121441e-14
685   WALLY2.0  ethereum  1.068039e-14
686        COQ  ethereum  4.436810e-15
687        NTC  ethereum  3.386478e-23

[688 rows x 3 columns]
Saved to portfolio.json
